# Implemtation of the graph

- Inicializar el grafo vacio NetworkX
- Crear un nodo. Para inicializar ese nodo buscamos en google (Primer resultado)
- N engines (computacion en paralelo) que busquen en los links
- Filtros URLs, Filtros Organizaciones (Revisar) - GPT2 (Clustering)
- Criterio de parada del agente.
- https://spacy.io/api/entityrecognizer


In [2]:
# Connection libraries.
import urllib.request, urllib.parse, urllib.error
import requests
from bs4 import BeautifulSoup
#from langdetect import detect
import re
import hashlib 
import time
import langdetect

import spacy
import es_core_news_sm
import en_core_web_sm
import networkx as nx

In [3]:
# Import class TheWitness (webscraper)
%run ./TheWitness.ipynb

# Import class Node 
# %run ./Node.ipynb

In [5]:
G = nx.Graph()
witness= TheWitness("https://co.mileroticos.com/escorts/bogota/buscar-servicio/", "https://co.mileroticos.com/escorts/bogota/buscar-servicio/", 
                    1, 3)

In [6]:
# For link in links.
def explore_network(result):
    
    count = 1
    for link in result[2]: 

        
        # We already explored this website.
        try: 
            if hashes_map[link]: 
                pass

        # New website!
        except:
            
            navegable = True
            for website in omit_pages: 
                if link.find(website) != -1: 
                    navegable = False
                    

            if navegable: 
                root = "https://modelosalacarta.com/"
                destination = link
                hash_destination = nodeID(destination)
                distance_from_root = 1
                #hashes_map[destination] = True

                witness = TheWitness(
                    root, 
                    destination, 
                    hash_destination, 
                    distance_from_root
                )
                result = witness.scrap()
                
                print("---------------")
                print("Agent No.", count)
                print(result)
                print("---------------")
                
                del witness
                
        count += 1

In [7]:
def nodeID(url): 
    m = hashlib.sha256()
    m.update(url.encode())
    return m.hexdigest()